In [1]:

import os
import json
import functools
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
df = pd.read_csv(r"D:\shalini\HS_DataClean_Nov.csv")

In [3]:
# Step 1: Read the config.json file
with open("D:\\Healthscore Codes\\healthscore_dashboard.json", 'r') as file:
    config_data = json.load(file)
    print(config_data)

{'features': [{'feature_name': 'OVER ALL', 'feature_seq_num': '1', 'metrics': [{'metrics': 'DR_Visits', 'display_names': 'Visits', 'metric_nature': 'Positive', 'metric_sequence_num': '1'}, {'metrics': 'Unique Visitors', 'display_names': 'Visitors', 'metric_nature': 'Positive', 'metric_sequence_num': '2'}, {'metrics': 'Page Views', 'display_names': 'Page Views', 'metric_nature': 'Positive', 'metric_sequence_num': '3'}, {'metrics': 'Average Time Spent on Page', 'display_names': 'Average Time Spent (mins)', 'metric_nature': 'Negative', 'metric_sequence_num': '4'}, {'metrics': 'DR_New Visitors', 'display_names': 'New Visitors', 'metric_nature': 'Positive', 'metric_sequence_num': '5'}, {'metrics': 'DR_Server Errors', 'display_names': 'Server Errors', 'metric_nature': 'Negative', 'metric_sequence_num': '6'}, {'metrics': 'DR_Overall Chat', 'display_names': 'Visits to Chat', 'metric_nature': 'Negative', 'metric_sequence_num': '7'}, {'metrics': 'DR_Message Us', 'display_names': 'Visits to Chat 

In [4]:
# Function to parse the nested dictionary
def parse_data(data):
    feature_groups = {}
    feature_seqno_mapping = {}
    display_name_mapping = {}
    metric_nature_mapping = {}
    metric_sequence_number_mapping = {}

    # Loop through the features in the data
    for feature in data['features']:
        feature_name = feature['feature_name']
        feature_seq_num = feature['feature_seq_num']
        
        # Add feature to feature groups
        feature_groups[feature_name] = []
        
        # Add feature seq no mapping
        feature_seqno_mapping[feature_name] = feature_seq_num
        
        for metric in feature['metrics']:
            metric_name = metric['metrics']
            display_name = metric['display_names']
            metric_nature = metric['metric_nature']
            metric_seq_num = metric['metric_sequence_num']
            
            # Add display name mapping
            display_name_mapping[metric_name] = display_name
            
            # Add metric nature mapping
            metric_nature_mapping[display_name] = metric_nature
            
            # Add metric sequence number mapping
            metric_sequence_number_mapping[display_name] = metric_seq_num
            
            # Add the metric name to the respective feature group
            feature_groups[feature_name].append(metric_name)

    return feature_groups, feature_seqno_mapping, display_name_mapping, metric_nature_mapping, metric_sequence_number_mapping

In [5]:
# Parse the data and get the mappings
feature_groups, feature_seqno_mapping, display_name_mapping, metric_nature_mapping, metric_sequence_number_mapping = parse_data(config_data)

In [6]:
# Step 2: Extract config1 and config2 from the file
level_1_metrics = config_data['level_1_metrics']
level_2_metrics = config_data['level_2_metrics']
level_3_metrics = config_data['level_3_metrics']
level_4_metrics = config_data['level_4_metrics']
level_5_metrics = config_data['level_5_metrics']


In [7]:
# Print the results (optional)
print("Feature Groups =", feature_groups)
print("Feature Seqno Mapping =", feature_seqno_mapping)
print("Display Name Mapping =", display_name_mapping)
print("Metric Nature Mapping =", metric_nature_mapping)
print("Metric Sequence Number Mapping =", metric_sequence_number_mapping)
print("level_1_metrics = ", level_1_metrics)
print("level_2_metrics = ", level_2_metrics)
print("level_3_metrics = ", level_3_metrics)
print("level_4_metrics = ", level_4_metrics)
print("level_5_metrics = ", level_5_metrics)

Feature Groups = {'OVER ALL': ['DR_Visits', 'Unique Visitors', 'Page Views', 'Average Time Spent on Page', 'DR_New Visitors', 'DR_Server Errors', 'DR_Overall Chat', 'DR_Message Us', 'DR_Agent Chat', 'DR_Self Service Initiated', 'DR_CoxApp | Visits with Chat and No Self Service Initiated', 'DR_CoxApp | Visits with No Self Service and No Chat', 'DR_CoxApp | Visits not logged in', 'DR_continuing chat from new chat page'], 'LOG IN': ['DR_Login Page Visits', 'DR_Successful logins', 'DR_Auto Logins', 'DR_Clicks to MFA Verification', 'DR_Clicks on Forgot User ID', 'Clicks on Forgot Password', 'DR_Need to Register'], 'BIL LING': ['DR_Visits to Billing Home', 'DR_Billing Make a Payment', 'DR_Payment Submit', 'DR_Payments', 'DR_Client Errors on Make Payment', 'DR_Server Errors on Make Payment', 'DR_Promise to Pay', 'DR_Promise to Pay (Native)', 'DR_CoxApp | Billing - Additional time to pay (Link out)', 'DR_CoxApp | Billing - Future date payment (Link out)', 'DR_Billing - View Bill Details', 'DR_

In [8]:
df['display_names'] = df['Metrics'].map(display_name_mapping)

In [9]:
df['metric_nature'] = df['display_names'].map(metric_nature_mapping)

In [10]:
df['level_1'] = df['display_names'].apply(lambda x: x if x in level_1_metrics else None)
df['level_2'] = df['display_names'].apply(lambda x: x if x in level_2_metrics else None)
df['level_3'] = df['display_names'].apply(lambda x: x if x in level_3_metrics else None)
df['level_4'] = df['display_names'].apply(lambda x: x if x in level_4_metrics else None)
df['level_5'] = df['display_names'].apply(lambda x: x if x in level_5_metrics else None)

In [11]:
feature_column = 'Unknown'
df['Feature'] = feature_column
for group, metrics in feature_groups.items():
   df['Feature'] = np.where(df['Metrics'].isin(metrics), group, df['Feature'])

In [12]:
def move_row(df, from_idx, to_idx):
    row = df.iloc[from_idx]
    df = df.drop(from_idx).reset_index(drop=True)
    df = pd.concat([df.iloc[:to_idx], row.to_frame().T, df.iloc[to_idx:]]).reset_index(drop=True)
    return df

df = move_row(df, 50, 46)
df = move_row(df, 51, 47)
df = move_row(df, 52, 48)
df = move_row(df, 53, 49)
df = move_row(df, 164, 158)
df = move_row(df, 165, 159)
df = move_row(df, 180, 174)
df = move_row(df, 181, 175)
df = move_row(df, 218, 206)
df = move_row(df, 219, 207)
df = move_row(df, 220, 208)
df = move_row(df, 221, 209)
df = move_row(df, 280, 268)
df = move_row(df, 281, 269)
df = move_row(df, 282, 270)
df = move_row(df, 283, 271)

In [13]:
df['unique_identifier'] = df['Feature'].apply(lambda x: ''.join([word[0].upper() for word in x.split()]))

In [14]:
def generate_hierarchy_id_grouped_by_os(df):
    df['Hierarchy_ID'] = None

   
    df['Feature_Order'] = df.groupby('Operating System Type')['unique_identifier'].transform(lambda x: pd.Series(range(len(x)), index=x.index))

    grouped = df.groupby(['Operating System Type', 'unique_identifier'], group_keys=False)
    results = []

   
    for (os_name, unique_id), group in grouped:
        current_ids = {'level_1': 0, 'level_2': 0, 'level_3': 0, 'level_4': 0, 'level_5': 0}  

        group = group.sort_values('Feature_Order')  

        for index, row in group.iterrows():
            if pd.isna(row['display_names']):
                continue  

            if not pd.isna(row['level_1']):  
                current_ids['level_1'] += 1
                current_ids['level_2'] = 0  
                current_ids['level_3'] = 0  
                current_ids['level_4'] = 0  
                current_ids['level_5'] = 0  
                group.at[index, 'Hierarchy_ID'] = f"{os_name}_{unique_id}_{current_ids['level_1']}"

            elif not pd.isna(row['level_2']):  
                current_ids['level_2'] += 1
                current_ids['level_3'] = 0  
                current_ids['level_4'] = 0  
                current_ids['level_5'] = 0  
                group.at[index, 'Hierarchy_ID'] = f"{os_name}_{unique_id}_{current_ids['level_1']}.{current_ids['level_2']}"

            elif not pd.isna(row['level_3']):  
                current_ids['level_3'] += 1
                current_ids['level_4'] = 0  
                current_ids['level_5'] = 0  
                group.at[index, 'Hierarchy_ID'] = f"{os_name}_{unique_id}_{current_ids['level_1']}.{current_ids['level_2']}.{current_ids['level_3']}"

            elif not pd.isna(row['level_4']):  
                current_ids['level_4'] += 1
                current_ids['level_5'] = 0  
                group.at[index, 'Hierarchy_ID'] = f"{os_name}_{unique_id}_{current_ids['level_1']}.{current_ids['level_2']}.{current_ids['level_3']}.{current_ids['level_4']}"

            elif not pd.isna(row['level_5']):  
                current_ids['level_5'] += 1
                group.at[index, 'Hierarchy_ID'] = f"{os_name}_{unique_id}_{current_ids['level_1']}.{current_ids['level_2']}.{current_ids['level_3']}.{current_ids['level_4']}.{current_ids['level_5']}"

        results.append(group)

   
    df = pd.concat(results, ignore_index=True)

    
    #df = df[df['Display_Names'].notna()]

    
    cols = [col for col in df.columns if col != 'Hierarchy_ID' and col != 'Feature_Order'] + ['Hierarchy_ID']
    return df[cols]


df = generate_hierarchy_id_grouped_by_os(df)


In [15]:
df = generate_hierarchy_id_grouped_by_os(df)

def assign_parent_id(row):
    if not pd.isna(row['level_5']):  
        return f"{row['Hierarchy_ID'].rsplit('.', 1)[0]}"
    elif not pd.isna(row['level_4']):  
        return f"{row['Hierarchy_ID'].rsplit('.', 1)[0]}"
    elif not pd.isna(row['level_3']):  
        return f"{row['Hierarchy_ID'].rsplit('.', 1)[0]}"
    elif not pd.isna(row['level_2']):  
        return f"{row['Hierarchy_ID'].rsplit('.', 1)[0]}"
    elif not pd.isna(row['level_1']): 
        return None
    else:
        return None


df['Parent_ID'] = df.apply(assign_parent_id, axis=1)


In [16]:
df['metric_id'] = range(1, len(df) + 1)

In [17]:
df['metric_sequence_num'] = df.groupby('Operating System Type').cumcount() + 1

In [18]:
df['metric_seqno'] = df['display_names'].map(metric_sequence_number_mapping)

In [19]:
def calculate_level_no(row):
   
    if pd.isna(row['Hierarchy_ID']) or pd.isna(row['Parent_ID']):
        return 1
    
   
    return row['Hierarchy_ID'].count('.') + 1


df['level_no'] = df.apply(calculate_level_no, axis=1)


In [20]:
feature_names = df['Feature'].unique()
feature_mapping = {feature_name: idx + 1 for idx, feature_name in enumerate(feature_names)}

df['feature_id'] = df['Feature'].map(feature_mapping)


In [21]:
# Create 'feature_seq_num' column based on the dictionary
df['feature_seq_num'] = df['Feature'].map(feature_seqno_mapping)

In [22]:
df['create_dt'] = pd.to_datetime('today').normalize().date()

In [23]:
df = df.rename(columns={'Parent_ID': 'parent_id_old'})

In [24]:
def calculate_parent_id(row, df):
    
    if pd.isna(row['Hierarchy_ID']):
        return None
   
    hierarchy_parts = row['Hierarchy_ID'].rsplit('.', 1)
    if len(hierarchy_parts) > 1:
       
        parent_hierarchy = hierarchy_parts[0]
        
        parent_row = df[df['Hierarchy_ID'] == parent_hierarchy]
        if not parent_row.empty:
            return parent_row['metric_id'].values[0]

    return None

df['parent_id'] = df.apply(calculate_parent_id, axis=1, df=df)


df.loc[df['Hierarchy_ID'].isna(), 'parent_id'] = None


In [25]:
df = df.rename(columns={
    'Metrics': 'metrics',
    'Operating System Type': 'operating_system_type',
    'Feature': 'feature_name',
    'Operating System Type': 'operating_system_type',
    'Feature': 'feature_name',
    'Operating System Type': 'operating_system_type',
    'Hierarchy_ID': 'hierarchy_id',
})

In [26]:
df.shape

(463, 111)

In [27]:
print(list(df.columns))

['metrics', 'operating_system_type', '29-08-2024', '30-08-2024', '31-08-2024', '01-09-2024', '02-09-2024', '03-09-2024', '04-09-2024', '05-09-2024', '06-09-2024', '07-09-2024', '08-09-2024', '09-09-2024', '10-09-2024', '11-09-2024', '12-09-2024', '13-09-2024', '14-09-2024', '15-09-2024', '16-09-2024', '17-09-2024', '18-09-2024', '19-09-2024', '20-09-2024', '21-09-2024', '22-09-2024', '23-09-2024', '24-09-2024', '25-09-2024', '26-09-2024', '27-09-2024', '28-09-2024', '29-09-2024', '30-09-2024', '01-10-2024', '02-10-2024', '03-10-2024', '04-10-2024', '05-10-2024', '06-10-2024', '07-10-2024', '08-10-2024', '09-10-2024', '10-10-2024', '11-10-2024', '12-10-2024', '13-10-2024', '14-10-2024', '15-10-2024', '16-10-2024', '17-10-2024', '18-10-2024', '19-10-2024', '20-10-2024', '21-10-2024', '22-10-2024', '23-10-2024', '24-10-2024', '25-10-2024', '26-10-2024', '27-10-2024', '28-10-2024', '29-10-2024', '30-10-2024', '31-10-2024', '01-11-2024', '02-11-2024', '03-11-2024', '04-11-2024', '05-11-2024

In [28]:
first_table_columns = ['metric_id', 'metrics', 'display_names', 'operating_system_type', 'feature_name', 'unique_identifier', 'level_1', 'level_2', 'level_3', 'level_4','level_5', 'hierarchy_id', 'parent_id_old', 'metric_sequence_num', 'metric_seqno', 'level_no', 'feature_id', 'feature_seq_num', 'parent_id', 'metric_nature', 'create_dt' ]
first_table_df = df[first_table_columns]

In [29]:
#first_table_df.to_csv(r'D:\Healthsore Data\table_1_data_all_features.csv', index = False)

In [30]:
def is_date_column(col_name):
    date_formats = ['%d-%m-%Y']
    for fmt in date_formats:
        try:
            pd.to_datetime(col_name, format=fmt)
            return True
        except ValueError:
            continue  # Try the next format
    return False

# Filter columns with date format 'm/d/Y'
date_columns = [col for col in df.columns if is_date_column(col)]

In [31]:
second_table_columns1 = ['metric_id', 'create_dt', 'metrics', 'operating_system_type']
second_table_columns = second_table_columns1 + date_columns
second_table_df = df[second_table_columns]

In [32]:
filtered_date_columns = second_table_df.columns[4:].to_list()

In [33]:
id_vars = ['metric_id', 'create_dt']

In [34]:
second_table_df_melted = second_table_df.melt(id_vars = id_vars, value_vars = filtered_date_columns, var_name = 'Date', value_name = 'Value')

In [35]:
second_table_df_melted.shape

(41670, 4)

In [36]:
#second_table_df_melted.to_csv(r'D:\Healthsore Data\Table2.csv', index = False)

In [37]:
df.dtypes

metrics                   object
operating_system_type     object
29-08-2024                object
30-08-2024                object
31-08-2024                object
                          ...   
level_no                   int64
feature_id                 int64
feature_seq_num           object
create_dt                 object
parent_id                float64
Length: 111, dtype: object

In [37]:
sample_df = df[df['feature_name'].isin(['SMART HELP', 'RESET MODEM', 'RESET TV BOX', 'LOG IN', 'BIL LING', 'OVER ALL', 'EASY CON NECT', 'DATA USAGE', 'RE GISTRATION (NATIVE)'])]
sample_df.shape

(218, 111)

In [38]:
def calculate_percentage(df, datecolumns, metricid, parentid):
    # Ensure these columns exist in the DataFrame
    if metricid not in df.columns or parentid not in df.columns:
        raise ValueError(f"'{metricid}' or '{parentid}' columns not found in DataFrame.")
    
    # Create a copy of the DataFrame to avoid setting on a view
    df = df.copy()
    
    # Iterate through each row and calculate the percentage for each DATE column
    for idx, row in df.iterrows():
        # Iterate through all the DATE columns
        for date_col in datecolumns:
            # Get the numerator (DATE value)
            numerator = row[date_col]
            
            # Get the parent hierarchy ID
            parent_id = row[parentid]
            
            # Check if the parent is empty
            if pd.isna(parent_id) or parent_id == "":
                # If the PARENTID is empty, keep the original DATE value
                percentage = numerator  # Assuming 100% when no parent is found
            else:
                # Find the row where HIERARCHYID == parent_hierarchyid to get the parent's DATE value
                parent_row = df[df[metricid] == parent_id]
                
                # If the parent exists, get the parent's DATE value for the current DATE column
                if not parent_row.empty:
                    denominator = parent_row[date_col].values[0]
                else:
                    denominator = 0  # In case no parent is found, avoid division by zero
                
                # Calculate the percentage
                if denominator != 0:
                    percentage = (numerator / denominator) * 100
                else:
                    percentage = 0  # To handle division by zero if no parent found
                
            # Round to 3 decimal places
            percentage = round(percentage, 3)
            
            # Add the percentage to the DataFrame in a new column
            percentage_column_name = f"{date_col} PERCENTAGE"
            if percentage_column_name not in df.columns:
                df[percentage_column_name] = None  # Initialize the column safely
            
            # Set the percentage for the current row and current date column using .loc
            df.loc[idx, percentage_column_name] = percentage

    return df


In [39]:
percentage_df = calculate_percentage(df, date_columns, 'metric_id', 'parent_id')
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 100)
percentage_df.head()

,metrics,operating_system_type,29-08-2024,30-08-2024,31-08-2024,01-09-2024,02-09-2024,03-09-2024,04-09-2024,05-09-2024,06-09-2024,07-09-2024,08-09-2024,09-09-2024,10-09-2024,11-09-2024,12-09-2024,13-09-2024,14-09-2024,15-09-2024,16-09-2024,17-09-2024,18-09-2024,19-09-2024,20-09-2024,21-09-2024,22-09-2024,23-09-2024,24-09-2024,25-09-2024,26-09-2024,27-09-2024,28-09-2024,29-09-2024,30-09-2024,01-10-2024,02-10-2024,03-10-2024,04-10-2024,05-10-2024,06-10-2024,07-10-2024,08-10-2024,09-10-2024,10-10-2024,11-10-2024,12-10-2024,13-10-2024,14-10-2024,15-10-2024,16-10-2024,17-10-2024,18-10-2024,19-10-2024,20-10-2024,21-10-2024,22-10-2024,23-10-2024,24-10-2024,25-10-2024,26-10-2024,27-10-2024,28-10-2024,29-10-2024,30-10-2024,31-10-2024,01-11-2024,02-11-2024,03-11-2024,04-11-2024,05-11-2024,06-11-2024,07-11-2024,08-11-2024,09-11-2024,10-11-2024,11-11-2024,12-11-2024,13-11-2024,14-11-2024,15-11-2024,16-11-2024,17-11-2024,18-11-2024,19-11-2024,20-11-2024,21-11-2024,22-11-2024,23-11-2024,24-11-2024,25-11-2024,26-11-2024,display_names,metric_nature,level_1,level_2,level_3,level_4,level_5,feature_name,unique_identifier,hierarchy_id,parent_id_old,metric_id,metric_sequence_num,metric_seqno,level_no,feature_id,feature_seq_num,create_dt,parent_id,29-08-2024 PERCENTAGE,30-08-2024 PERCENTAGE,31-08-2024 PERCENTAGE,01-09-2024 PERCENTAGE,02-09-2024 PERCENTAGE,03-09-2024 PERCENTAGE,04-09-2024 PERCENTAGE,05-09-2024 PERCENTAGE,06-09-2024 PERCENTAGE,07-09-2024 PERCENTAGE,08-09-2024 PERCENTAGE,09-09-2024 PERCENTAGE,10-09-2024 PERCENTAGE,11-09-2024 PERCENTAGE,12-09-2024 PERCENTAGE,13-09-2024 PERCENTAGE,14-09-2024 PERCENTAGE,15-09-2024 PERCENTAGE,16-09-2024 PERCENTAGE,17-09-2024 PERCENTAGE,18-09-2024 PERCENTAGE,19-09-2024 PERCENTAGE,20-09-2024 PERCENTAGE,21-09-2024 PERCENTAGE,22-09-2024 PERCENTAGE,23-09-2024 PERCENTAGE,24-09-2024 PERCENTAGE,25-09-2024 PERCENTAGE,26-09-2024 PERCENTAGE,27-09-2024 PERCENTAGE,28-09-2024 PERCENTAGE,29-09-2024 PERCENTAGE,30-09-2024 PERCENTAGE,01-10-2024 PERCENTAGE,02-10-2024 PERCENTAGE,03-10-2024 PERCENTAGE,04-10-2024 PERCENTAGE,05-10-2024 PERCENTAGE,06-10-2024 PERCENTAGE,07-10-2024 PERCENTAGE,08-10-2024 PERCENTAGE,09-10-2024 PERCENTAGE,10-10-2024 PERCENTAGE,11-10-2024 PERCENTAGE,12-10-2024 PERCENTAGE,13-10-2024 PERCENTAGE,14-10-2024 PERCENTAGE,15-10-2024 PERCENTAGE,16-10-2024 PERCENTAGE,17-10-2024 PERCENTAGE,18-10-2024 PERCENTAGE,19-10-2024 PERCENTAGE,20-10-2024 PERCENTAGE,21-10-2024 PERCENTAGE,22-10-2024 PERCENTAGE,23-10-2024 PERCENTAGE,24-10-2024 PERCENTAGE,25-10-2024 PERCENTAGE,26-10-2024 PERCENTAGE,27-10-2024 PERCENTAGE,28-10-2024 PERCENTAGE,29-10-2024 PERCENTAGE,30-10-2024 PERCENTAGE,31-10-2024 PERCENTAGE,01-11-2024 PERCENTAGE,02-11-2024 PERCENTAGE,03-11-2024 PERCENTAGE,04-11-2024 PERCENTAGE,05-11-2024 PERCENTAGE,06-11-2024 PERCENTAGE,07-11-2024 PERCENTAGE,08-11-2024 PERCENTAGE,09-11-2024 PERCENTAGE,10-11-2024 PERCENTAGE,11-11-2024 PERCENTAGE,12-11-2024 PERCENTAGE,13-11-2024 PERCENTAGE,14-11-2024 PERCENTAGE,15-11-2024 PERCENTAGE,16-11-2024 PERCENTAGE,17-11-2024 PERCENTAGE,18-11-2024 PERCENTAGE,19-11-2024 PERCENTAGE,20-11-2024 PERCENTAGE,21-11-2024 PERCENTAGE,22-11-2024 PERCENTAGE,23-11-2024 PERCENTAGE,24-11-2024 PERCENTAGE,25-11-2024 PERCENTAGE,26-11-2024 PERCENTAGE
0,DR_Visits to Billing Home,Apple iOS,71742.0,76236.0,54132.0,52596.0,54447.0,68414.0,70355.0,79002.0,78610.0,50904.0,47500.0,59885.0,60599.0,66866.0,75881.0,78958.0,52352.0,48231.0,60364.0,57000.0,63905.0,64500.0,71753.0,49102.0,46906.0,58086.0,61623.0,65367.0,64902.0,74267.0,56022.0,49393.0,69232.0,77165.0,66744.0,73629.0,73180.0,53129.0,47358.0,57406.0,58604.0,65363.0,70718.0,73814.0,50310.0,42931.0,59605.0,62231.0,65649.0,67679.0,73278.0,49194.0,46554.0,58498.0,60820.0,61559.0,64139.0,66553.0,45236.0,40903.0,53844.0,55602.0,62178.0,65295.0,80962.0,55882.0,53873.0,60230.0,61201.0,61253.0,69508.0,73999.0,50243.0,43774.0,54835.0,56204.0,64712.0,75234.0,81910.0,50202.0,45884.0,57871.0,58212.0,65289.0,60150.0,69650.0,48374.0,43920.0,58120.0,59662.0,Billing Section 

In [39]:
percentage_df.shape

(463, 201)

In [40]:
id_columns = ['create_dt', 'metric_id', 'feature_id', 'feature_seq_num', 'metric_sequence_num', 'metric_seqno','level_no', 'parent_id', 'unique_identifier', 'feature_name', 'metrics', 'display_names', 'operating_system_type', 'metric_nature']

In [41]:
# List of columns that contain "PERCENTAGE"
percentage_columns = [col for col in percentage_df.columns if 'PERCENTAGE' in col]

In [42]:
percent_df = percentage_df[id_columns + percentage_columns]

In [43]:
# Rename columns to remove the word "PERCENTAGE"
percent_df.columns = [col.replace(" PERCENTAGE", "") if "PERCENTAGE" in col else col for col in percent_df.columns]

In [44]:
percent_df.shape

(463, 104)

In [45]:
percent_df.head()

,create_dt,metric_id,feature_id,feature_seq_num,metric_sequence_num,metric_seqno,level_no,parent_id,unique_identifier,feature_name,metrics,display_names,operating_system_type,metric_nature,29-08-2024,30-08-2024,31-08-2024,01-09-2024,02-09-2024,03-09-2024,04-09-2024,05-09-2024,06-09-2024,07-09-2024,08-09-2024,09-09-2024,10-09-2024,11-09-2024,12-09-2024,13-09-2024,14-09-2024,15-09-2024,16-09-2024,17-09-2024,18-09-2024,19-09-2024,20-09-2024,21-09-2024,22-09-2024,23-09-2024,24-09-2024,25-09-2024,26-09-2024,27-09-2024,28-09-2024,29-09-2024,30-09-2024,01-10-2024,02-10-2024,03-10-2024,04-10-2024,05-10-2024,06-10-2024,07-10-2024,08-10-2024,09-10-2024,10-10-2024,11-10-2024,12-10-2024,13-10-2024,14-10-2024,15-10-2024,16-10-2024,17-10-2024,18-10-2024,19-10-2024,20-10-2024,21-10-2024,22-10-2024,23-10-2024,24-10-2024,25-10-2024,26-10-2024,27-10-2024,28-10-2024,29-10-2024,30-10-2024,31-10-2024,01-11-2024,02-11-2024,03-11-2024,04-11-2024,05-11-2024,06-11-2024,07-11-2024,08-11-2024,09-11-2024,10-11-2024,11-11-2024,12-11-2024,13-11-2024,14-11-2024,15-11-2024,16-11-2024,17-11-2024,18-11-2024,19-11-2024,20-11-2024,21-11-2024,22-11-2024,23-11-2024,24-11-2024,25-11-2024,26-11-2024
0,2025-02-06,1,1,3,1,1,1,NaN,BL,BIL LING,DR_Visits to Billing Home,Billing Section Visits,Apple iOS,Positive,71742.0,76236.0,54132.0,52596.0,54447.0,68414.0,70355.0,79002.0,78610.0,50904.0,47500.0,59885.0,60599.0,66866.0,75881.0,78958.0,52352.0,48231.0,60364.0,57000.0,63905.0,64500.0,71753.0,49102.0,46906.0,58086.0,61623.0,65367.0,64902.0,74267.0,56022.0,49393.0,69232.0,77165.0,66744.0,73629.0,73180.0,53129.0,47358.0,57406.0,58604.0,65363.0,70718.0,73814.0,50310.0,42931.0,59605.0,62231.0,65649.0,67679.0,73278.0,49194.0,46554.0,58498.0,60820.0,61559.0,64139.0,66553.0,45236.0,40903.0,53844.0,55602.0,62178.0,65295.0,80962.0,55882.0,53873.0,60230.0,61201.0,61253.0,69508.0,73999.0,50243.0,43774.0,54835.0,56204.0,64712.0,75234.0,81910.0,50202.0,45884.0,57871.0,58212.0,65289.0,60150.0,69650.0,48374.0,43920.0,58120.0,59662.0
1,2025-02-06,2,1,3,2,2,2,1.0,BL,BIL LING,DR_Billing Make a Payment,Make Payments,Apple iOS,Positive,39.419,43.499,37.287,36.775,34.426,33.727,36.383,40.112,44.15,34.797,33.366,33.616,34.85,38.247,40.542,46.293,37.254,35.23,34.206,33.795,36.201,39.623,43.802,35.459,34.578,33.168,33.101,36.107,38.709,41.861,37.869,35.112,35.078,38.011,37.516,40.449,44.269,37.688,36.03,34.909,35.492,38.652,42.855,48.124,40.302,39.559,35.739,36.602,39.516,42.353,45.578,38.332,36.785,35.406,34.931,38.207,40.648,43.618,36.175,34.543,32.637,33.925,38.961,44.553,47.148,39.131,36.358,35.446,36.658,39.195,43.04,46.359,38.306,37.186,33.907,33.909,39.021,42.745,46.442,37.831,35.982,34.577,35.439,38.907,39.862,44.307,38.167,36.004,33.73,36.827
2,2025-02-06,3,1,3,3,5,3,2.0,BL,BIL LING,DR_Client Errors on Make Payment,Client Errors on Payment Page,Apple iOS,Negative,0.004,0.0,0.005,0.005,0.0,0.0,0.0,0.0,0.0,0.006,0.0,0.0,0.0,0.0,0.0,0.003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005,0.0,0.004,0.0,0.0,0.0,0.0,0.003,0.0,0.0,0.0,0.0,0.0,0.0,0.005,0.0,0.0,0.003,0.0,0.0,0.0,0.004,0.0,0.0,0.0,0.0,0.0,0.005,0.0,0.0,0.0,0.003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005,0.0,0.0,0.004,0.0,0.0,0.0,0.0,0.0,0.0,0.004,0.003,0.0,0.005,0.0,0.0,0.0,0.004,0.004,0.0,0.0,0.006,0.0,0.0
3,2025-02-06,4,1,3,4,6,3,2.0,BL,BIL LING,DR_Server Errors on Make Payment,Server Errors on Payment Page,Apple iOS,Negative,0.004,0.009,0.015,0.01,0.0,0.004,0.012,0.006,0.012,0.0,0.0,0.01,0.005,0.0,0.003,0.003,0.01,0.0,0.0,0.005,0.004,0.016,0.016,0.006,0.0,0.005,0.025,0.0,0.008,0.0,0.009,0.012,0.0,0.01,0.004,0.007,0.012,0.015,0.006,0.0,0.01,0.0,0.0,0.003,0.01,0.018,0.005,0.022,0.008,0.014,0.009,0.016,0.023,0.024,0.0,0.004,0.008,0.017,0.012,0.0,0.0,0.005,0.008,0.0,0.003,0.014,0.005,0.014,0.004,0.0,0.0,0.003,0.005,0.006,0.011,0.021,0.0,0.006,0.003,0.011,0.012,0.0,0.005,0.012,0.017,0.01,0.005,0.0,0.0,0.018
4,2025-02-06,5,1,3,5,3,2,1.0,BL,BIL LING,DR_Payment Submit,Payment Submit,Apple iOS,Positive,36.536,40.807,35.134,34.611,31.822,30.67

In [46]:
def calculate_both(df, metricid, parentid):
    """
    Generate new rows for each unique display_name based on given conditions.

    Args:
    - df (DataFrame): Input DataFrame containing the original data.

    Returns:
    - DataFrame: Updated DataFrame with newly calculated rows.
    """
    new_rows = []  # Store new rows
    seq_num = 1  # Sequential ID for metric_id

    for display_name in df['display_names'].unique():
        # Filter rows for the current display_name
        display_df = df[df['display_names'] == display_name]
            
        # Case 1: If `parent_id_old` has null or empty values
        if display_df[parentid].isnull().any() or display_df[parentid].eq('').any():
            # Sum of `Apple iOS` and `Google Android` for `parent_id_old` being null
            ios_rows = display_df[display_df["operating_system_type"] == "Apple iOS"]
            android_rows = display_df[display_df["operating_system_type"] == "Google Android"]
            
            # Calculate numerator using `where` to avoid downcasting issues
            ios_sums = ios_rows[date_columns].apply(pd.to_numeric, errors='coerce').reset_index(drop=True)
            android_sums = android_rows[date_columns].apply(pd.to_numeric, errors='coerce').reset_index(drop=True)
            numerator = ios_sums + android_sums
            
            # Store numerator in new row, rounded to 3 decimal places
            new_row_values = numerator.sum(axis=0).fillna(0).apply(lambda x: round(x, 3)).to_dict()
        else:
            # Case 2: If `parent_id_old` is not null
            ios_rows = display_df[(display_df["operating_system_type"] == "Apple iOS") & 
                                  (display_df[parentid].notna()) & 
                                  (display_df[parentid] != '')]
            android_rows = display_df[(display_df["operating_system_type"] == "Google Android") & 
                                      (display_df[parentid].notna()) & 
                                      (display_df[parentid] != '')]
            
            # Calculate numerator using `where` to avoid downcasting issues
            ios_sums = ios_rows[date_columns].apply(pd.to_numeric, errors='coerce').reset_index(drop=True)
            android_sums = android_rows[date_columns].apply(pd.to_numeric, errors='coerce').reset_index(drop=True)
            numerator = ios_sums + android_sums
            
            # Filter rows where `parent_id_old` matches `hierarchy_id`
            filtered_data = df[df[metricid].isin(display_df[parentid])]
            ios_data = filtered_data[filtered_data['operating_system_type'] == "Apple iOS"]
            android_data = filtered_data[filtered_data['operating_system_type'] == "Google Android"]

            # Calculate denominator
            ios_sums_denom = ios_data[date_columns].apply(pd.to_numeric, errors='coerce').reset_index(drop=True)
            android_sums_denom = android_data[date_columns].apply(pd.to_numeric, errors='coerce').reset_index(drop=True)
            denominator = ios_sums_denom + android_sums_denom

            # Calculate result as percentage
            result = (numerator / denominator) * 100 if not denominator.empty else pd.DataFrame(0, columns=date_columns, index=numerator.index)
            result = result.round(3)  # Round result to 3 decimal places
            new_row_values = result.sum(axis=0).fillna(0).apply(lambda x: round(x, 3)).to_dict()

        # Create the new row for the current display_name
        new_row = {
            'display_names': display_name,
            'operating_system_type': 'Both',
            'metric_id': f'100000{seq_num}'
        }
        # Add calculated values to the new row
        new_row.update(new_row_values)
        new_rows.append(new_row)
        seq_num += 1

    # Append new rows to the original DataFrame
    new_rows_df = pd.DataFrame(new_rows)
    result_df = pd.concat([df, new_rows_df], ignore_index=True)

    # Reset index
    result_df.reset_index(drop=True, inplace=True)
    return new_rows_df


In [47]:
both_calc_df = calculate_both(df,'metric_id', 'parent_id')
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 300)      # Show 100 rows
both_calc_df.head(5)

C:\Users\gokul.02\AppData\Local\Temp\ipykernel_24404\1546731256.py:30: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  new_row_values = numerator.sum(axis=0).fillna(0).apply(lambda x: round(x, 3)).to_dict()
C:\Users\gokul.02\AppData\Local\Temp\ipykernel_24404\1546731256.py:30: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  new_row_values = numerator.sum(axis=0).fillna(0).apply(lambda x: round(x, 3)).to_dict()
C:\Users\gokul.02\AppData\Local\Temp\ipykernel_24404\1546731256.py:30: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is de

,display_names,operating_system_type,metric_id,29-08-2024,30-08-2024,31-08-2024,01-09-2024,02-09-2024,03-09-2024,04-09-2024,05-09-2024,06-09-2024,07-09-2024,08-09-2024,09-09-2024,10-09-2024,11-09-2024,12-09-2024,13-09-2024,14-09-2024,15-09-2024,16-09-2024,17-09-2024,18-09-2024,19-09-2024,20-09-2024,21-09-2024,22-09-2024,23-09-2024,24-09-2024,25-09-2024,26-09-2024,27-09-2024,28-09-2024,29-09-2024,30-09-2024,01-10-2024,02-10-2024,03-10-2024,04-10-2024,05-10-2024,06-10-2024,07-10-2024,08-10-2024,09-10-2024,10-10-2024,11-10-2024,12-10-2024,13-10-2024,14-10-2024,15-10-2024,16-10-2024,17-10-2024,18-10-2024,19-10-2024,20-10-2024,21-10-2024,22-10-2024,23-10-2024,24-10-2024,25-10-2024,26-10-2024,27-10-2024,28-10-2024,29-10-2024,30-10-2024,31-10-2024,01-11-2024,02-11-2024,03-11-2024,04-11-2024,05-11-2024,06-11-2024,07-11-2024,08-11-2024,09-11-2024,10-11-2024,11-11-2024,12-11-2024,13-11-2024,14-11-2024,15-11-2024,16-11-2024,17-11-2024,18-11-2024,19-11-2024,20-11-2024,21-11-2024,22-11-2024,23-11-2024,24-11-2024,25-11-2024,26-11-2024
0,Billing Section Visits,Both,1000001,93064.000,99690.000,70846.000,68476.000,70777.000,89457.000,91825.000,102981.000,102789.000,66804.000,61675.000,77547.000,78493.000,86857.000,97900.000,102689.000,68247.000,63041.000,77821.000,73506.000,82820.000,83588.000,93325.000,63873.000,60709.000,74423.000,79316.000,85112.000,84457.000,96598.000,72554.000,63698.000,89212.000,101101.000,87238.000,97925.000,96309.000,69725.000,61658.000,74383.000,76292.000,85505.000,92385.000,96333.000,65765.000,55555.000,76985.000,80619.000,85310.000,88308.000,95437.000,64272.000,60384.000,76148.000,79110.000,79923.000,83441.000,86559.000,59192.000,53204.000,69949.000,72277.000,80683.000,85194.000,107366.000,73527.000,70262.000,78308.000,79580.000,80010.000,90858.000,96496.000,65867.000,57008.000,70899.000,72458.000,84140.000,97421.000,106566.000,65666.000,59653.000,74638.000,75190.000,84863.000,79320.000,91369.000,63344.000,56984.000,74961.000,77062.000
1,Make Payments,Both,1000002,39.606,44.064,37.685,36.988,34.547,34.328,36.616,40.437,44.384,34.913,33.561,33.578,34.942,38.425,40.869,46.502,37.508,35.141,34.211,33.784,36.433,39.903,44.020,35.538,34.632,33.025,33.010,36.374,38.726,42.087,37.558,34.992,35.308,38.815,37.707,41.385,44.555,37.698,35.867,35.040,35.479,38.856,43.070,48.501,40.190,39.424,35.586,36.350,39.715,42.450,45.719,38.139,36.596,35.199,34.814,38.253,40.838,43.697,36.285,34.640,33.157,34.059,39.326,44.588,48.098,39.479,36.464,35.653,36.678,39.403,43.290,46.531,38.359,37.126,33.855,33.854,39.324,42.852,46.429,37.971,35.968,34.436,35.372,38.999,40.205,44.483,38.076,35.800,33.743,36.827
2,Client Errors on Payment Page,Both,1000003,0.003,0.000,0.004,0.004,0.000,0.000,0.000,0.000,0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.004,0.000,0.003,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.004,0.000,0.000,0.002,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.004,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.004,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.003,0.002,0.000,0.004,0.000,0.000,0.000,0.003,0.003,0.000,0.000,0.005,0.000,0.000
3,Server Errors on Payment Page,Both,1000004,0.263,0.355,0.360,0.276,0.344,0.374,0.404,0.358,0.379,0.412,0.367,0.338,0.368,0.318,0.315,0.285,0.359,0.316,0.316,0.455,0.335,0.315,0.277,0.326,0.366,0.309,0.439,0.355,0.321,0.275,0.349,0.319,0.333,0.385,0.365,0.358,0.329,0.380,0.348,0.315,0.351,0.331,0.279,0.274,0.299,0.356,0.318,0.358,0.342,0.269,0.252,0.396,0.385,0.351,0.305,0.383,0.302,0.336,0.396,0.336,0.375,0.284,0.334,0.329,0.354,0.365,0.324,0.380,0.291,0.333,0.353,0.314,0.336,0.378,0.308,0.314,0.287,0.275,0.249,0.361,0.354,0.315,0.323,0.299,0.376,0.335,0.328,0.284,0.265,0.296
4,Payment Submit,Both,1000005,36.462,41.049,34.924,34.270,31.465,30.872,33.069,37.242,41.623,32.492,30.623,29.973,31.421,35.098,37.783,43.567,34.500,32.344,30.771,30.814,33.442,36.806,41.259

In [48]:
both_calc_df.shape

(251, 93)

In [49]:
unique_display_names_df_test = df.drop_duplicates(subset='display_names')
unique_display_names_df_test.shape

(251, 111)

In [50]:
id_columns_both_test = ['create_dt', 'feature_id', 'feature_seq_num', 'metric_sequence_num', 'metric_seqno', 'parent_id', 'unique_identifier', 'feature_name', 'metrics', 'display_names','metric_nature' ]
id_column_both_df_test = unique_display_names_df_test[id_columns_both_test]
#id_column_both_df = id_column_both_df[id_column_both_df['feature_name'].isin(['SMART HELP'])]
id_column_both_df_test.shape

(251, 11)

In [51]:
# Perform a left merge on 'display_names'
both_df_test =  pd.merge(both_calc_df, id_column_both_df_test, on='display_names', how='inner')
both_df_test.shape

(251, 103)

In [52]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 300)      # Show 100 rows
both_df_test.head()

,display_names,operating_system_type,metric_id,29-08-2024,30-08-2024,31-08-2024,01-09-2024,02-09-2024,03-09-2024,04-09-2024,05-09-2024,06-09-2024,07-09-2024,08-09-2024,09-09-2024,10-09-2024,11-09-2024,12-09-2024,13-09-2024,14-09-2024,15-09-2024,16-09-2024,17-09-2024,18-09-2024,19-09-2024,20-09-2024,21-09-2024,22-09-2024,23-09-2024,24-09-2024,25-09-2024,26-09-2024,27-09-2024,28-09-2024,29-09-2024,30-09-2024,01-10-2024,02-10-2024,03-10-2024,04-10-2024,05-10-2024,06-10-2024,07-10-2024,08-10-2024,09-10-2024,10-10-2024,11-10-2024,12-10-2024,13-10-2024,14-10-2024,15-10-2024,16-10-2024,17-10-2024,18-10-2024,19-10-2024,20-10-2024,21-10-2024,22-10-2024,23-10-2024,24-10-2024,25-10-2024,26-10-2024,27-10-2024,28-10-2024,29-10-2024,30-10-2024,31-10-2024,01-11-2024,02-11-2024,03-11-2024,04-11-2024,05-11-2024,06-11-2024,07-11-2024,08-11-2024,09-11-2024,10-11-2024,11-11-2024,12-11-2024,13-11-2024,14-11-2024,15-11-2024,16-11-2024,17-11-2024,18-11-2024,19-11-2024,20-11-2024,21-11-2024,22-11-2024,23-11-2024,24-11-2024,25-11-2024,26-11-2024,create_dt,feature_id,feature_seq_num,metric_sequence_num,metric_seqno,parent_id,unique_identifier,feature_name,metrics,metric_nature
0,Billing Section Visits,Both,1000001,93064.000,99690.000,70846.000,68476.000,70777.000,89457.000,91825.000,102981.000,102789.000,66804.000,61675.000,77547.000,78493.000,86857.000,97900.000,102689.000,68247.000,63041.000,77821.000,73506.000,82820.000,83588.000,93325.000,63873.000,60709.000,74423.000,79316.000,85112.000,84457.000,96598.000,72554.000,63698.000,89212.000,101101.000,87238.000,97925.000,96309.000,69725.000,61658.000,74383.000,76292.000,85505.000,92385.000,96333.000,65765.000,55555.000,76985.000,80619.000,85310.000,88308.000,95437.000,64272.000,60384.000,76148.000,79110.000,79923.000,83441.000,86559.000,59192.000,53204.000,69949.000,72277.000,80683.000,85194.000,107366.000,73527.000,70262.000,78308.000,79580.000,80010.000,90858.000,96496.000,65867.000,57008.000,70899.000,72458.000,84140.000,97421.000,106566.000,65666.000,59653.000,74638.000,75190.000,84863.000,79320.000,91369.000,63344.000,56984.000,74961.000,77062.000,2025-02-06,1,3,1,1,NaN,BL,BIL LING,DR_Visits to Billing Home,Positive
1,Make Payments,Both,1000002,39.606,44.064,37.685,36.988,34.547,34.328,36.616,40.437,44.384,34.913,33.561,33.578,34.942,38.425,40.869,46.502,37.508,35.141,34.211,33.784,36.433,39.903,44.020,35.538,34.632,33.025,33.010,36.374,38.726,42.087,37.558,34.992,35.308,38.815,37.707,41.385,44.555,37.698,35.867,35.040,35.479,38.856,43.070,48.501,40.190,39.424,35.586,36.350,39.715,42.450,45.719,38.139,36.596,35.199,34.814,38.253,40.838,43.697,36.285,34.640,33.157,34.059,39.326,44.588,48.098,39.479,36.464,35.653,36.678,39.403,43.290,46.531,38.359,37.126,33.855,33.854,39.324,42.852,46.429,37.971,35.968,34.436,35.372,38.999,40.205,44.483,38.076,35.800,33.743,36.827,2025-02-06,1,3,2,2,1.0,BL,BIL LING,DR_Billing Make a Payment,Positive
2,Client Errors on Payment Page,Both,1000003,0.003,0.000,0.004,0.004,0.000,0.000,0.000,0.000,0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.004,0.000,0.003,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.004,0.000,0.000,0.002,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.004,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.004,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.003,0.002,0.000,0.004,0.000,0.000,0.000,0.003,0.003,0.000,0.000,0.005,0.000,0.000,2025-02-06,1,3,3,5,2.0,BL,BIL LING,DR_Client Errors on Make Payment,Negative
3,Server Errors on Payment Page,Both,1000004,0.263,0.355,0.360,0.276,0.344,0.374,0.404,0.358,0.379,0.412,0.367,0.338,0.368,0.318,0.315,0.285,0.359,0.316,0.316,0.455,0.335,0.315,0.277,0.326,0.366,0.309,0.439,0.355,0.321,0.275,0.349,0.319,0.333,0.385,0.365,0.358,0.329,0.380,0.348,0.315,0.351,0.331,0.279,0.274,0.299,0.356,0.318,0.358,0.342,0.269,0.252,0.396,0.385,0.351,0.305,0.383,0.302,0.336,0.396,0.336,0.375,0.284,0.334,0.

In [53]:
print(list(both_df_test))

['display_names', 'operating_system_type', 'metric_id', '29-08-2024', '30-08-2024', '31-08-2024', '01-09-2024', '02-09-2024', '03-09-2024', '04-09-2024', '05-09-2024', '06-09-2024', '07-09-2024', '08-09-2024', '09-09-2024', '10-09-2024', '11-09-2024', '12-09-2024', '13-09-2024', '14-09-2024', '15-09-2024', '16-09-2024', '17-09-2024', '18-09-2024', '19-09-2024', '20-09-2024', '21-09-2024', '22-09-2024', '23-09-2024', '24-09-2024', '25-09-2024', '26-09-2024', '27-09-2024', '28-09-2024', '29-09-2024', '30-09-2024', '01-10-2024', '02-10-2024', '03-10-2024', '04-10-2024', '05-10-2024', '06-10-2024', '07-10-2024', '08-10-2024', '09-10-2024', '10-10-2024', '11-10-2024', '12-10-2024', '13-10-2024', '14-10-2024', '15-10-2024', '16-10-2024', '17-10-2024', '18-10-2024', '19-10-2024', '20-10-2024', '21-10-2024', '22-10-2024', '23-10-2024', '24-10-2024', '25-10-2024', '26-10-2024', '27-10-2024', '28-10-2024', '29-10-2024', '30-10-2024', '31-10-2024', '01-11-2024', '02-11-2024', '03-11-2024', '04-11

In [54]:
both_df_test.shape

(251, 103)

In [55]:
modifiled_first_table_df = pd.concat([df, both_df_test], ignore_index=True)

In [56]:
modifiled_first_table_df.shape

(714, 111)

In [57]:
print(list(modifiled_first_table_df))

['metrics', 'operating_system_type', '29-08-2024', '30-08-2024', '31-08-2024', '01-09-2024', '02-09-2024', '03-09-2024', '04-09-2024', '05-09-2024', '06-09-2024', '07-09-2024', '08-09-2024', '09-09-2024', '10-09-2024', '11-09-2024', '12-09-2024', '13-09-2024', '14-09-2024', '15-09-2024', '16-09-2024', '17-09-2024', '18-09-2024', '19-09-2024', '20-09-2024', '21-09-2024', '22-09-2024', '23-09-2024', '24-09-2024', '25-09-2024', '26-09-2024', '27-09-2024', '28-09-2024', '29-09-2024', '30-09-2024', '01-10-2024', '02-10-2024', '03-10-2024', '04-10-2024', '05-10-2024', '06-10-2024', '07-10-2024', '08-10-2024', '09-10-2024', '10-10-2024', '11-10-2024', '12-10-2024', '13-10-2024', '14-10-2024', '15-10-2024', '16-10-2024', '17-10-2024', '18-10-2024', '19-10-2024', '20-10-2024', '21-10-2024', '22-10-2024', '23-10-2024', '24-10-2024', '25-10-2024', '26-10-2024', '27-10-2024', '28-10-2024', '29-10-2024', '30-10-2024', '31-10-2024', '01-11-2024', '02-11-2024', '03-11-2024', '04-11-2024', '05-11-2024

In [58]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 500)      # Show 100 rows
modifiled_first_table_df = modifiled_first_table_df[['metric_id', 'metrics', 'display_names', 'operating_system_type', 'feature_name', 'unique_identifier', 'metric_nature', 'level_1', 'level_2', 'level_3', 'level_4', 'level_5', 'hierarchy_id', 'parent_id_old', 'metric_sequence_num', 'metric_seqno', 'level_no', 'feature_id', 'feature_seq_num', 'parent_id', 'create_dt']]
modifiled_first_table_df.head()

,metric_id,metrics,display_names,operating_system_type,feature_name,unique_identifier,metric_nature,level_1,level_2,level_3,level_4,level_5,hierarchy_id,parent_id_old,metric_sequence_num,metric_seqno,level_no,feature_id,feature_seq_num,parent_id,create_dt
0,1,DR_Visits to Billing Home,Billing Section Visits,Apple iOS,BIL LING,BL,Positive,Billing Section Visits,None,None,None,None,Apple iOS_BL_1,None,1,1,1.0,1,3,NaN,2025-02-06
1,2,DR_Billing Make a Payment,Make Payments,Apple iOS,BIL LING,BL,Positive,None,Make Payments,None,None,None,Apple iOS_BL_1.1,Apple iOS_BL_1,2,2,2.0,1,3,1.0,2025-02-06
2,3,DR_Client Errors on Make Payment,Client Errors on Payment Page,Apple iOS,BIL LING,BL,Negative,None,None,Client Errors on Payment Page,None,None,Apple iOS_BL_1.1.1,Apple iOS_BL_1.1,3,5,3.0,1,3,2.0,2025-02-06
3,4,DR_Server Errors on Make Payment,Server Errors on Payment Page,Apple iOS,BIL LING,BL,Negative,None,None,Server Errors on Payment Page,None,None,Apple iOS_BL_1.1.2,Apple iOS_BL_1.1,4,6,3.0,1,3,2.0,2025-02-06
4,5,DR_Payment Submit,Payment Submit,Apple iOS,BIL LING,BL,Positive,None,Payment Submit,None,None,None,Apple iOS_BL_1.2,Apple iOS_BL_1,5,3,2.0,1,3,1.0,2025-02-06


In [59]:
#modifiled_first_table_df.to_csv(r'D:\Healthsore Data\Table1.csv', index = False)

In [60]:
unique_display_names_df = df.drop_duplicates(subset='display_names')
unique_display_names_df.shape

(251, 111)

In [61]:
id_columns_both = ['create_dt', 'feature_id', 'feature_seq_num', 'metric_sequence_num', 'metric_seqno', 'parent_id', 'unique_identifier', 'feature_name', 'metrics', 'display_names','metric_nature' ]
id_column_both_df = unique_display_names_df[id_columns_both]
#id_column_both_df = id_column_both_df[id_column_both_df['feature_name'].isin(['SMART HELP'])]
id_column_both_df.shape

(251, 11)

In [62]:
# Perform a left merge on 'display_names'
both_df =  pd.merge(both_calc_df, id_column_both_df, on='display_names', how='inner')
both_df.shape

(251, 103)

In [63]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 300)      # Show 100 rows
both_df.head()


,display_names,operating_system_type,metric_id,29-08-2024,30-08-2024,31-08-2024,01-09-2024,02-09-2024,03-09-2024,04-09-2024,05-09-2024,06-09-2024,07-09-2024,08-09-2024,09-09-2024,10-09-2024,11-09-2024,12-09-2024,13-09-2024,14-09-2024,15-09-2024,16-09-2024,17-09-2024,18-09-2024,19-09-2024,20-09-2024,21-09-2024,22-09-2024,23-09-2024,24-09-2024,25-09-2024,26-09-2024,27-09-2024,28-09-2024,29-09-2024,30-09-2024,01-10-2024,02-10-2024,03-10-2024,04-10-2024,05-10-2024,06-10-2024,07-10-2024,08-10-2024,09-10-2024,10-10-2024,11-10-2024,12-10-2024,13-10-2024,14-10-2024,15-10-2024,16-10-2024,17-10-2024,18-10-2024,19-10-2024,20-10-2024,21-10-2024,22-10-2024,23-10-2024,24-10-2024,25-10-2024,26-10-2024,27-10-2024,28-10-2024,29-10-2024,30-10-2024,31-10-2024,01-11-2024,02-11-2024,03-11-2024,04-11-2024,05-11-2024,06-11-2024,07-11-2024,08-11-2024,09-11-2024,10-11-2024,11-11-2024,12-11-2024,13-11-2024,14-11-2024,15-11-2024,16-11-2024,17-11-2024,18-11-2024,19-11-2024,20-11-2024,21-11-2024,22-11-2024,23-11-2024,24-11-2024,25-11-2024,26-11-2024,create_dt,feature_id,feature_seq_num,metric_sequence_num,metric_seqno,parent_id,unique_identifier,feature_name,metrics,metric_nature
0,Billing Section Visits,Both,1000001,93064.000,99690.000,70846.000,68476.000,70777.000,89457.000,91825.000,102981.000,102789.000,66804.000,61675.000,77547.000,78493.000,86857.000,97900.000,102689.000,68247.000,63041.000,77821.000,73506.000,82820.000,83588.000,93325.000,63873.000,60709.000,74423.000,79316.000,85112.000,84457.000,96598.000,72554.000,63698.000,89212.000,101101.000,87238.000,97925.000,96309.000,69725.000,61658.000,74383.000,76292.000,85505.000,92385.000,96333.000,65765.000,55555.000,76985.000,80619.000,85310.000,88308.000,95437.000,64272.000,60384.000,76148.000,79110.000,79923.000,83441.000,86559.000,59192.000,53204.000,69949.000,72277.000,80683.000,85194.000,107366.000,73527.000,70262.000,78308.000,79580.000,80010.000,90858.000,96496.000,65867.000,57008.000,70899.000,72458.000,84140.000,97421.000,106566.000,65666.000,59653.000,74638.000,75190.000,84863.000,79320.000,91369.000,63344.000,56984.000,74961.000,77062.000,2025-02-06,1,3,1,1,NaN,BL,BIL LING,DR_Visits to Billing Home,Positive
1,Make Payments,Both,1000002,39.606,44.064,37.685,36.988,34.547,34.328,36.616,40.437,44.384,34.913,33.561,33.578,34.942,38.425,40.869,46.502,37.508,35.141,34.211,33.784,36.433,39.903,44.020,35.538,34.632,33.025,33.010,36.374,38.726,42.087,37.558,34.992,35.308,38.815,37.707,41.385,44.555,37.698,35.867,35.040,35.479,38.856,43.070,48.501,40.190,39.424,35.586,36.350,39.715,42.450,45.719,38.139,36.596,35.199,34.814,38.253,40.838,43.697,36.285,34.640,33.157,34.059,39.326,44.588,48.098,39.479,36.464,35.653,36.678,39.403,43.290,46.531,38.359,37.126,33.855,33.854,39.324,42.852,46.429,37.971,35.968,34.436,35.372,38.999,40.205,44.483,38.076,35.800,33.743,36.827,2025-02-06,1,3,2,2,1.0,BL,BIL LING,DR_Billing Make a Payment,Positive
2,Client Errors on Payment Page,Both,1000003,0.003,0.000,0.004,0.004,0.000,0.000,0.000,0.000,0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.004,0.000,0.003,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.004,0.000,0.000,0.002,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.004,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.004,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.003,0.002,0.000,0.004,0.000,0.000,0.000,0.003,0.003,0.000,0.000,0.005,0.000,0.000,2025-02-06,1,3,3,5,2.0,BL,BIL LING,DR_Client Errors on Make Payment,Negative
3,Server Errors on Payment Page,Both,1000004,0.263,0.355,0.360,0.276,0.344,0.374,0.404,0.358,0.379,0.412,0.367,0.338,0.368,0.318,0.315,0.285,0.359,0.316,0.316,0.455,0.335,0.315,0.277,0.326,0.366,0.309,0.439,0.355,0.321,0.275,0.349,0.319,0.333,0.385,0.365,0.358,0.329,0.380,0.348,0.315,0.351,0.331,0.279,0.274,0.299,0.356,0.318,0.358,0.342,0.269,0.252,0.396,0.385,0.351,0.305,0.383,0.302,0.336,0.396,0.336,0.375,0.284,0.334,0.

In [64]:
third_table_df = pd.concat([percent_df, both_df], ignore_index=True)

In [65]:
third_table_df.shape

(714, 104)

In [66]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 200)      # Show 100 rows
third_table_df.head()

,create_dt,metric_id,feature_id,feature_seq_num,metric_sequence_num,metric_seqno,level_no,parent_id,unique_identifier,feature_name,metrics,display_names,operating_system_type,metric_nature,29-08-2024,30-08-2024,31-08-2024,01-09-2024,02-09-2024,03-09-2024,04-09-2024,05-09-2024,06-09-2024,07-09-2024,08-09-2024,09-09-2024,10-09-2024,11-09-2024,12-09-2024,13-09-2024,14-09-2024,15-09-2024,16-09-2024,17-09-2024,18-09-2024,19-09-2024,20-09-2024,21-09-2024,22-09-2024,23-09-2024,24-09-2024,25-09-2024,26-09-2024,27-09-2024,28-09-2024,29-09-2024,30-09-2024,01-10-2024,02-10-2024,03-10-2024,04-10-2024,05-10-2024,06-10-2024,07-10-2024,08-10-2024,09-10-2024,10-10-2024,11-10-2024,12-10-2024,13-10-2024,14-10-2024,15-10-2024,16-10-2024,17-10-2024,18-10-2024,19-10-2024,20-10-2024,21-10-2024,22-10-2024,23-10-2024,24-10-2024,25-10-2024,26-10-2024,27-10-2024,28-10-2024,29-10-2024,30-10-2024,31-10-2024,01-11-2024,02-11-2024,03-11-2024,04-11-2024,05-11-2024,06-11-2024,07-11-2024,08-11-2024,09-11-2024,10-11-2024,11-11-2024,12-11-2024,13-11-2024,14-11-2024,15-11-2024,16-11-2024,17-11-2024,18-11-2024,19-11-2024,20-11-2024,21-11-2024,22-11-2024,23-11-2024,24-11-2024,25-11-2024,26-11-2024
0,2025-02-06,1,1,3,1,1,1.0,NaN,BL,BIL LING,DR_Visits to Billing Home,Billing Section Visits,Apple iOS,Positive,71742.0,76236.0,54132.0,52596.0,54447.0,68414.0,70355.0,79002.0,78610.0,50904.0,47500.0,59885.0,60599.0,66866.0,75881.0,78958.0,52352.0,48231.0,60364.0,57000.0,63905.0,64500.0,71753.0,49102.0,46906.0,58086.0,61623.0,65367.0,64902.0,74267.0,56022.0,49393.0,69232.0,77165.0,66744.0,73629.0,73180.0,53129.0,47358.0,57406.0,58604.0,65363.0,70718.0,73814.0,50310.0,42931.0,59605.0,62231.0,65649.0,67679.0,73278.0,49194.0,46554.0,58498.0,60820.0,61559.0,64139.0,66553.0,45236.0,40903.0,53844.0,55602.0,62178.0,65295.0,80962.0,55882.0,53873.0,60230.0,61201.0,61253.0,69508.0,73999.0,50243.0,43774.0,54835.0,56204.0,64712.0,75234.0,81910.0,50202.0,45884.0,57871.0,58212.0,65289.0,60150.0,69650.0,48374.0,43920.0,58120.0,59662.0
1,2025-02-06,2,1,3,2,2,2.0,1.0,BL,BIL LING,DR_Billing Make a Payment,Make Payments,Apple iOS,Positive,39.419,43.499,37.287,36.775,34.426,33.727,36.383,40.112,44.15,34.797,33.366,33.616,34.85,38.247,40.542,46.293,37.254,35.23,34.206,33.795,36.201,39.623,43.802,35.459,34.578,33.168,33.101,36.107,38.709,41.861,37.869,35.112,35.078,38.011,37.516,40.449,44.269,37.688,36.03,34.909,35.492,38.652,42.855,48.124,40.302,39.559,35.739,36.602,39.516,42.353,45.578,38.332,36.785,35.406,34.931,38.207,40.648,43.618,36.175,34.543,32.637,33.925,38.961,44.553,47.148,39.131,36.358,35.446,36.658,39.195,43.04,46.359,38.306,37.186,33.907,33.909,39.021,42.745,46.442,37.831,35.982,34.577,35.439,38.907,39.862,44.307,38.167,36.004,33.73,36.827
2,2025-02-06,3,1,3,3,5,3.0,2.0,BL,BIL LING,DR_Client Errors on Make Payment,Client Errors on Payment Page,Apple iOS,Negative,0.004,0.0,0.005,0.005,0.0,0.0,0.0,0.0,0.0,0.006,0.0,0.0,0.0,0.0,0.0,0.003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005,0.0,0.004,0.0,0.0,0.0,0.0,0.003,0.0,0.0,0.0,0.0,0.0,0.0,0.005,0.0,0.0,0.003,0.0,0.0,0.0,0.004,0.0,0.0,0.0,0.0,0.0,0.005,0.0,0.0,0.0,0.003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005,0.0,0.0,0.004,0.0,0.0,0.0,0.0,0.0,0.0,0.004,0.003,0.0,0.005,0.0,0.0,0.0,0.004,0.004,0.0,0.0,0.006,0.0,0.0
3,2025-02-06,4,1,3,4,6,3.0,2.0,BL,BIL LING,DR_Server Errors on Make Payment,Server Errors on Payment Page,Apple iOS,Negative,0.004,0.009,0.015,0.01,0.0,0.004,0.012,0.006,0.012,0.0,0.0,0.01,0.005,0.0,0.003,0.003,0.01,0.0,0.0,0.005,0.004,0.016,0.016,0.006,0.0,0.005,0.025,0.0,0.008,0.0,0.009,0.012,0.0,0.01,0.004,0.007,0.012,0.015,0.006,0.0,0.01,0.0,0.0,0.003,0.01,0.018,0.005,0.022,0.008,0.014,0.009,0.016,0.023,0.024,0.0,0.004,0.008,0.017,0.012,0.0,0.0,0.005,0.008,0.0,0.003,0.014,0.005,0.014,0.004,0.0,0.0,0.003,0.005,0.006,0.011,0.021,0.0,0.006,0.003,0.011,0.012,0.0,0.005,0.012,0.017,0.01,0.005,0.0,0.0,0.018
4,2025-02-06,5,1,3,5,3,2.0,1.0,BL,BIL LING,DR_Payment Submit,Payment Submit,Apple iOS,Positive,36.536,40.807,35.134,34.611,31

In [67]:
print(list(third_table_df))

['create_dt', 'metric_id', 'feature_id', 'feature_seq_num', 'metric_sequence_num', 'metric_seqno', 'level_no', 'parent_id', 'unique_identifier', 'feature_name', 'metrics', 'display_names', 'operating_system_type', 'metric_nature', '29-08-2024', '30-08-2024', '31-08-2024', '01-09-2024', '02-09-2024', '03-09-2024', '04-09-2024', '05-09-2024', '06-09-2024', '07-09-2024', '08-09-2024', '09-09-2024', '10-09-2024', '11-09-2024', '12-09-2024', '13-09-2024', '14-09-2024', '15-09-2024', '16-09-2024', '17-09-2024', '18-09-2024', '19-09-2024', '20-09-2024', '21-09-2024', '22-09-2024', '23-09-2024', '24-09-2024', '25-09-2024', '26-09-2024', '27-09-2024', '28-09-2024', '29-09-2024', '30-09-2024', '01-10-2024', '02-10-2024', '03-10-2024', '04-10-2024', '05-10-2024', '06-10-2024', '07-10-2024', '08-10-2024', '09-10-2024', '10-10-2024', '11-10-2024', '12-10-2024', '13-10-2024', '14-10-2024', '15-10-2024', '16-10-2024', '17-10-2024', '18-10-2024', '19-10-2024', '20-10-2024', '21-10-2024', '22-10-2024',

In [68]:
id_columns = ['create_dt', 'metric_id', 'feature_id', 'feature_seq_num', 'metric_sequence_num', 'metric_seqno', 'level_no', 'parent_id', 'unique_identifier', 'feature_name', 'metrics', 'display_names', 'operating_system_type', 'metric_nature']

In [69]:
#third_table_df.to_csv(r'D:\Healthsore Data\Table3.csv', index = False)

In [70]:
third_table_melted_df = pd.melt(third_table_df, id_vars=id_columns, 
                     value_vars=date_columns, 
                     var_name='Date', 
                     value_name='Value')

In [71]:
third_table_melted_df.shape

(64260, 16)

In [72]:
third_table_melted_df.columns

Index(['create_dt', 'metric_id', 'feature_id', 'feature_seq_num',
       'metric_sequence_num', 'metric_seqno', 'level_no', 'parent_id',
       'unique_identifier', 'feature_name', 'metrics', 'display_names',
       'operating_system_type', 'metric_nature', 'Date', 'Value'],
      dtype='object')

In [73]:
third_table_final_df = third_table_melted_df[['create_dt', 'metric_id', 'feature_id', 'feature_name', 'metrics', 'display_names', 'operating_system_type', 'Date', 'Value']]

In [74]:
third_table_final_df.head()

,create_dt,metric_id,feature_id,feature_name,metrics,display_names,operating_system_type,Date,Value
0,2025-02-06,1,1,BIL LING,DR_Visits to Billing Home,Billing Section Visits,Apple iOS,29-08-2024,71742.0
1,2025-02-06,2,1,BIL LING,DR_Billing Make a Payment,Make Payments,Apple iOS,29-08-2024,39.419
2,2025-02-06,3,1,BIL LING,DR_Client Errors on Make Payment,Client Errors on Payment Page,Apple iOS,29-08-2024,0.004
3,2025-02-06,4,1,BIL LING,DR_Server Errors on Make Payment,Server Errors on Payment Page,Apple iOS,29-08-2024,0.004
4,2025-02-06,5,1,BIL LING,DR_Payment Submit,Payment Submit,Apple iOS,29-08-2024,36.536


In [75]:
third_table_final_df.shape

(64260, 9)

In [76]:
#third_table_final_df.to_csv(r'D:\Healthsore Data\Table3_athena.csv', index = False)

In [77]:
third_table_melted_df.dtypes

create_dt                 object
metric_id                 object
feature_id                 int64
feature_seq_num           object
metric_sequence_num        int64
metric_seqno              object
level_no                 float64
parent_id                float64
unique_identifier         object
feature_name              object
metrics                   object
display_names             object
operating_system_type     object
metric_nature             object
Date                      object
Value                     object
dtype: object

In [78]:
# Convert 'create_dt' and 'Date' to datetime
third_table_melted_df['create_dt'] = pd.to_datetime(third_table_melted_df['create_dt'], errors='coerce')
third_table_melted_df['Date'] = pd.to_datetime(third_table_melted_df['Date'], errors='coerce')

# Convert 'feature_id', 'metric_sequence_num', 'level_no', 'parent_id' to integers
third_table_melted_df['feature_id'] = third_table_melted_df['feature_id'].astype('Int64')  # 'Int64' to handle missing values (NaN)
third_table_melted_df['feature_seq_num'] = third_table_melted_df['feature_seq_num'].astype('Int64')
third_table_melted_df['metric_sequence_num'] = third_table_melted_df['metric_sequence_num'].astype('Int64')
#third_table_melted_df['metric_seqno'] = third_table_melted_df['metric_seqno'].astype('Int64')
third_table_melted_df['level_no'] = third_table_melted_df['level_no'].astype('Int64')
third_table_melted_df['parent_id'] = third_table_melted_df['parent_id'].astype('Int64')
#third_table_melted_df['metric_id'] = third_table_melted_df['metric_id'].astype('Int64')

# Convert 'metric_id', 'unique_identifier', 'feature_name', 'metrics', 'display_names', 'operating_system_type' to strings
third_table_melted_df['metric_id'] = third_table_melted_df['metric_id'].astype(str)
third_table_melted_df['unique_identifier'] = third_table_melted_df['unique_identifier'].astype(str)
third_table_melted_df['feature_name'] = third_table_melted_df['feature_name'].astype(str)
third_table_melted_df['metrics'] = third_table_melted_df['metrics'].astype(str)
third_table_melted_df['display_names'] = third_table_melted_df['display_names'].astype(str)
third_table_melted_df['operating_system_type'] = third_table_melted_df['operating_system_type'].astype(str)
third_table_melted_df['metric_nature'] = third_table_melted_df['metric_nature'].astype(str)

# Convert 'Value' to float
third_table_melted_df['Value'] = pd.to_numeric(third_table_melted_df['Value'], errors='coerce')

C:\Users\gokul.02\AppData\Local\Temp\ipykernel_24404\3155458354.py:3: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  third_table_melted_df['Date'] = pd.to_datetime(third_table_melted_df['Date'], errors='coerce')


In [79]:
third_table_melted_df.dtypes

create_dt                datetime64[ns]
metric_id                        object
feature_id                        Int64
feature_seq_num                   Int64
metric_sequence_num               Int64
metric_seqno                     object
level_no                          Int64
parent_id                         Int64
unique_identifier                object
feature_name                     object
metrics                          object
display_names                    object
operating_system_type            object
metric_nature                    object
Date                     datetime64[ns]
Value                           float64
dtype: object

In [80]:
#Calculate 1st and 3rd quartiles
quartiles = third_table_melted_df.groupby(['operating_system_type', 'display_names'])['Value'].quantile([0.25, 0.75]).unstack()
quartiles.reset_index(inplace=True)
quartiles.columns.name = None  # Remove the index name
quartiles.columns = ['operating_system_type', 'display_names', '1st Quartile', '3rd Quartile']
#Calculate IQR
quartiles['IQR'] = quartiles['3rd Quartile'] - quartiles['1st Quartile']

In [81]:
quartiles.shape

(708, 5)

In [82]:
quartiles_iqr_df = pd.merge(third_table_df, quartiles, on=['display_names', 'operating_system_type'], how='outer')

In [83]:
quartiles_iqr_df.shape

(714, 107)

In [84]:
pd.set_option('display.max_columns', None)  # Show all columns 
pd.set_option('display.max_rows', 200)      # Show 100 rows 
quartiles_iqr_df.head()

,create_dt,metric_id,feature_id,feature_seq_num,metric_sequence_num,metric_seqno,level_no,parent_id,unique_identifier,feature_name,metrics,display_names,operating_system_type,metric_nature,29-08-2024,30-08-2024,31-08-2024,01-09-2024,02-09-2024,03-09-2024,04-09-2024,05-09-2024,06-09-2024,07-09-2024,08-09-2024,09-09-2024,10-09-2024,11-09-2024,12-09-2024,13-09-2024,14-09-2024,15-09-2024,16-09-2024,17-09-2024,18-09-2024,19-09-2024,20-09-2024,21-09-2024,22-09-2024,23-09-2024,24-09-2024,25-09-2024,26-09-2024,27-09-2024,28-09-2024,29-09-2024,30-09-2024,01-10-2024,02-10-2024,03-10-2024,04-10-2024,05-10-2024,06-10-2024,07-10-2024,08-10-2024,09-10-2024,10-10-2024,11-10-2024,12-10-2024,13-10-2024,14-10-2024,15-10-2024,16-10-2024,17-10-2024,18-10-2024,19-10-2024,20-10-2024,21-10-2024,22-10-2024,23-10-2024,24-10-2024,25-10-2024,26-10-2024,27-10-2024,28-10-2024,29-10-2024,30-10-2024,31-10-2024,01-11-2024,02-11-2024,03-11-2024,04-11-2024,05-11-2024,06-11-2024,07-11-2024,08-11-2024,09-11-2024,10-11-2024,11-11-2024,12-11-2024,13-11-2024,14-11-2024,15-11-2024,16-11-2024,17-11-2024,18-11-2024,19-11-2024,20-11-2024,21-11-2024,22-11-2024,23-11-2024,24-11-2024,25-11-2024,26-11-2024,1st Quartile,3rd Quartile,IQR
0,2025-02-06,22,2,13,22,,1.0,NaN,CM,COX MOBILE,DR_Mobile Lock Enabled,,Apple iOS,,39.0,37.0,37.0,25.0,46.0,48.0,44.0,37.0,56.0,32.0,36.0,36.0,30.0,40.0,36.0,24.0,37.0,25.0,42.0,33.0,26.0,34.0,35.0,44.0,39.0,42.0,52.0,44.0,45.0,46.0,51.0,35.0,54.0,45.0,43.0,49.0,50.0,41.0,46.0,49.0,49.0,51.0,39.0,44.0,44.0,24.0,43.0,59.0,50.0,46.0,41.0,39.0,43.0,53.0,48.0,40.0,47.0,41.0,43.0,44.0,40.0,42.0,49.0,46.0,54.0,39.0,39.0,36.0,34.0,59.0,45.0,41.0,39.0,39.0,40.0,51.0,47.0,41.0,52.0,38.0,48.0,57.0,46.0,49.0,59.0,51.0,47.0,42.0,61.0,42.0,0.0,143.00,143.00
1,2025-02-06,23,2,13,23,,1.0,NaN,CM,COX MOBILE,DR_Mobile Lock Disabled,,Apple iOS,,512.0,497.0,446.0,422.0,423.0,531.0,547.0,563.0,516.0,448.0,389.0,562.0,564.0,587.0,597.0,571.0,506.0,481.0,551.0,547.0,582.0,549.0,691.0,600.0,536.0,669.0,659.0,718.0,694.0,673.0,574.0,532.0,620.0,616.0,609.0,669.0,647.0,620.0,500.0,608.0,591.0,597.0,610.0,598.0,533.0,460.0,649.0,633.0,615.0,621.0,608.0,551.0,529.0,652.0,650.0,629.0,640.0,585.0,539.0,474.0,584.0,616.0,591.0,541.0,580.0,493.0,501.0,615.0,529.0,558.0,585.0,562.0,532.0,461.0,570.0,566.0,583.0,636.0,606.0,518.0,461.0,610.0,639.0,656.0,616.0,651.0,627.0,525.0,679.0,648.0,0.0,143.00,143.00
2,2025-02-06,74,5,4,74,,1.0,NaN,ECN,EASY CON NECT,DR_Easy Connect - Self install success,,Apple iOS,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,143.00,143.00
3,2025-02-06,75,5,4,75,,1.0,NaN,ECN,EASY CON NECT,DR_Easy Connect - Self install Timeout,,Apple iOS,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,143.00,143.00
4,2025-02-06,10000022,2,13,22,,NaN,NaN,CM,COX MOBILE,DR_Mobile Lock Enabled,,Both,,782.0,790.0,708.0,607.0,663.0,829.0,858.0,856.0,836.0,723.0,609.0,825.0,814.0,879.0,894.0,844.0,745.0,685.0,816.0,804.0,853.0,800.0,993.0,896.0,766.0,937.0,984.0,1025.0,965.0,995.0,880.0,770.0,930.0,972.0,932.0,963.0,973.0,918.0,731.0,890.0,884.0,904.0,931.0,870.0,771.0,635.0,904.0,938.0,868.0,899.0,872.0,812.0,766.0,942.0,935.0,905.0,921.0,865.0,799.0,694.0,824.0,874.0,887.0,808.0,903.0,778.0,753.0,885.0,817.0,839.0,869.0,824.0,830.0,665.0,836.0,850.0,886.0,956.0,968.0,858.0,731.0,919.0,976.0,975.0,966.0,1029.0,1034.0,813.0,1006.0,1023.0,805.0,930.75,1

In [85]:

def calculate_upper_lower(df, parent_column='parent_id'):
    # Ensure parent_column is present in the dataframe
    if parent_column not in df.columns:
        raise ValueError(f"{parent_column} not found in the dataframe")
    
    # Define the function to apply to each row
    def calculate_row(row):
        if pd.isna(row[parent_column]) or row[parent_column] == '':
            # Apply 1.25 logic for rows where parent_id is null, empty, or None
            upper = row['3rd Quartile'] + (1.25 * row['IQR'])
            lower = row['1st Quartile'] - (1.25 * row['IQR'])
        else:
            # Apply 0.75 logic for other rows
            upper = row['3rd Quartile'] + (0.75 * row['IQR'])
            lower = row['1st Quartile'] - (0.75 * row['IQR'])
        
        return pd.Series({'Upper': upper, 'Lower': lower})
    
    # Apply the function row by row
    df[['Upper', 'Lower']] = df.apply(calculate_row, axis=1)

    # Round the Upper column to 1 decimal place
    #df['Upper'] = df['Upper'].round(1)

    return df


In [86]:
upper_lower_df = calculate_upper_lower(quartiles_iqr_df)
#upper_lower_df =upper_lower_df[['display_names','operating_system_type','1st Quartile','3rd Quartile', 'IQR', 'Upper', 'Lower']]
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 200)      # Show 100 rows
upper_lower_df.head()

,create_dt,metric_id,feature_id,feature_seq_num,metric_sequence_num,metric_seqno,level_no,parent_id,unique_identifier,feature_name,metrics,display_names,operating_system_type,metric_nature,29-08-2024,30-08-2024,31-08-2024,01-09-2024,02-09-2024,03-09-2024,04-09-2024,05-09-2024,06-09-2024,07-09-2024,08-09-2024,09-09-2024,10-09-2024,11-09-2024,12-09-2024,13-09-2024,14-09-2024,15-09-2024,16-09-2024,17-09-2024,18-09-2024,19-09-2024,20-09-2024,21-09-2024,22-09-2024,23-09-2024,24-09-2024,25-09-2024,26-09-2024,27-09-2024,28-09-2024,29-09-2024,30-09-2024,01-10-2024,02-10-2024,03-10-2024,04-10-2024,05-10-2024,06-10-2024,07-10-2024,08-10-2024,09-10-2024,10-10-2024,11-10-2024,12-10-2024,13-10-2024,14-10-2024,15-10-2024,16-10-2024,17-10-2024,18-10-2024,19-10-2024,20-10-2024,21-10-2024,22-10-2024,23-10-2024,24-10-2024,25-10-2024,26-10-2024,27-10-2024,28-10-2024,29-10-2024,30-10-2024,31-10-2024,01-11-2024,02-11-2024,03-11-2024,04-11-2024,05-11-2024,06-11-2024,07-11-2024,08-11-2024,09-11-2024,10-11-2024,11-11-2024,12-11-2024,13-11-2024,14-11-2024,15-11-2024,16-11-2024,17-11-2024,18-11-2024,19-11-2024,20-11-2024,21-11-2024,22-11-2024,23-11-2024,24-11-2024,25-11-2024,26-11-2024,1st Quartile,3rd Quartile,IQR,Upper,Lower
0,2025-02-06,22,2,13,22,,1.0,NaN,CM,COX MOBILE,DR_Mobile Lock Enabled,,Apple iOS,,39.0,37.0,37.0,25.0,46.0,48.0,44.0,37.0,56.0,32.0,36.0,36.0,30.0,40.0,36.0,24.0,37.0,25.0,42.0,33.0,26.0,34.0,35.0,44.0,39.0,42.0,52.0,44.0,45.0,46.0,51.0,35.0,54.0,45.0,43.0,49.0,50.0,41.0,46.0,49.0,49.0,51.0,39.0,44.0,44.0,24.0,43.0,59.0,50.0,46.0,41.0,39.0,43.0,53.0,48.0,40.0,47.0,41.0,43.0,44.0,40.0,42.0,49.0,46.0,54.0,39.0,39.0,36.0,34.0,59.0,45.0,41.0,39.0,39.0,40.0,51.0,47.0,41.0,52.0,38.0,48.0,57.0,46.0,49.0,59.0,51.0,47.0,42.0,61.0,42.0,0.0,143.00,143.00,321.7500,-178.7500
1,2025-02-06,23,2,13,23,,1.0,NaN,CM,COX MOBILE,DR_Mobile Lock Disabled,,Apple iOS,,512.0,497.0,446.0,422.0,423.0,531.0,547.0,563.0,516.0,448.0,389.0,562.0,564.0,587.0,597.0,571.0,506.0,481.0,551.0,547.0,582.0,549.0,691.0,600.0,536.0,669.0,659.0,718.0,694.0,673.0,574.0,532.0,620.0,616.0,609.0,669.0,647.0,620.0,500.0,608.0,591.0,597.0,610.0,598.0,533.0,460.0,649.0,633.0,615.0,621.0,608.0,551.0,529.0,652.0,650.0,629.0,640.0,585.0,539.0,474.0,584.0,616.0,591.0,541.0,580.0,493.0,501.0,615.0,529.0,558.0,585.0,562.0,532.0,461.0,570.0,566.0,583.0,636.0,606.0,518.0,461.0,610.0,639.0,656.0,616.0,651.0,627.0,525.0,679.0,648.0,0.0,143.00,143.00,321.7500,-178.7500
2,2025-02-06,74,5,4,74,,1.0,NaN,ECN,EASY CON NECT,DR_Easy Connect - Self install success,,Apple iOS,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,143.00,143.00,321.7500,-178.7500
3,2025-02-06,75,5,4,75,,1.0,NaN,ECN,EASY CON NECT,DR_Easy Connect - Self install Timeout,,Apple iOS,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,143.00,143.00,321.7500,-178.7500
4,2025-02-06,10000022,2,13,22,,NaN,NaN,CM,COX MOBILE,DR_Mobile Lock Enabled,,Both,,782.0,790.0,708.0,607.0,663.0,829.0,858.0,856.0,836.0,723.0,609.0,825.0,814.0,879.0,894.0,844.0,745.0,685.0,816.0,804.0,853.0,800.0,993.0,896.0,766.0,937.0,984.0,1025.0,965.0,995.0,880.0,770.0,930.0,972.0,932.0,963.0,973.0,918.0,731.0,890.0,884.0,904.0,931.0,870.0,771.0,635.0,904.0,938.0,868.0,899.0,872.0,812.0,766.0,942.0,935.0,905.0,921.0,865.0,799.0,694.0,824.0,874.0,887.0,808.0,903.0,778.0,753.0,885.0,817.0,839.0,869.0,824.0,830.0,665.0,836.0,850.0,886.0,956.0,96

In [87]:
upper_lower_df.shape

(714, 109)

In [88]:
import numpy as np
import pandas as pd

def calculate_last7_and_30_days(df):
    # Extract the date columns (skip the non-date columns like 'display_names' and 'operating_system_type')
    datecolumn = [col for col in df.columns if '-' in col]
    
    # Exclude the latest date column (the first one)
    last_7_columns = datecolumn[-8:-1]  # Get the last 7 columns excluding the latest date
    last_30_columns = datecolumn[-31:-1]

    # Extract the date part and convert them to datetime objects
    date_objects = [pd.to_datetime(col.split()[0], format='%d-%m-%Y') for col in datecolumn]

    # Get the column corresponding to the latest date
    latest_date = max(date_objects)
    latest_date_column = datecolumn[date_objects.index(latest_date)]

    # Create a new column 'Yesterday' with the values from the latest date column
    df.loc[:, 'Yesterday'] = df[latest_date_column]
    
    # Ensure numeric columns before performing mean calculation
    df[last_7_columns] = df[last_7_columns].apply(pd.to_numeric, errors='coerce')
    df[last_30_columns] = df[last_30_columns].apply(pd.to_numeric, errors='coerce')

    # Calculate the mean for each row across the last 7 and 30 date columns
    df.loc[:, 'last_7_days'] = df[last_7_columns].mean(axis=1)
    df.loc[:, 'last_30_days'] = df[last_30_columns].mean(axis=1)

    # Replace 0 and NaN values in 'last_7_days' and 'last_30_days' with NaN to avoid division by zero
    df['last_7_days'] = df['last_7_days'].replace(0, np.nan)
    df['last_30_days'] = df['last_30_days'].replace(0, np.nan)

    # Calculate the percentage change for 'Last 7 Days' with a check for NaN
    df.loc[:, '% Change Last 7 Days'] = np.where(
        df['last_7_days'].isna(), 0, 
        (df['Yesterday'] - df['last_7_days']) / df['last_7_days'] * 100
    )

    # Calculate the percentage change for 'Last 30 Days' with a check for NaN
    df.loc[:, '% Change Last 30 Days'] = np.where(
        df['last_30_days'].isna(), 0, 
        (df['Yesterday'] - df['last_30_days']) / df['last_30_days'] * 100
    )

    # Round the percentage changes to 1 decimal place
    df.loc[:, '% Change Last 7 Days'] = df['% Change Last 7 Days'].round(1)
    df.loc[:, '% Change Last 30 Days'] = df['% Change Last 30 Days'].round(1)
    
    return df


In [89]:
last_7_and_30_days_df  = calculate_last7_and_30_days(upper_lower_df)
#last_7_and_30_days_df = last_7_and_30_days_df[['metric_id', 'metrics', 'display_names', 'operating_system_type', 'feature_name', 'parent_id','Yesterday', 'last_7_days', 'last_30_days', '% Change Last 7 Days', '% Change Last 30 Days']]
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 100)
last_7_and_30_days_df.head()

,create_dt,metric_id,feature_id,feature_seq_num,metric_sequence_num,metric_seqno,level_no,parent_id,unique_identifier,feature_name,metrics,display_names,operating_system_type,metric_nature,29-08-2024,30-08-2024,31-08-2024,01-09-2024,02-09-2024,03-09-2024,04-09-2024,05-09-2024,06-09-2024,07-09-2024,08-09-2024,09-09-2024,10-09-2024,11-09-2024,12-09-2024,13-09-2024,14-09-2024,15-09-2024,16-09-2024,17-09-2024,18-09-2024,19-09-2024,20-09-2024,21-09-2024,22-09-2024,23-09-2024,24-09-2024,25-09-2024,26-09-2024,27-09-2024,28-09-2024,29-09-2024,30-09-2024,01-10-2024,02-10-2024,03-10-2024,04-10-2024,05-10-2024,06-10-2024,07-10-2024,08-10-2024,09-10-2024,10-10-2024,11-10-2024,12-10-2024,13-10-2024,14-10-2024,15-10-2024,16-10-2024,17-10-2024,18-10-2024,19-10-2024,20-10-2024,21-10-2024,22-10-2024,23-10-2024,24-10-2024,25-10-2024,26-10-2024,27-10-2024,28-10-2024,29-10-2024,30-10-2024,31-10-2024,01-11-2024,02-11-2024,03-11-2024,04-11-2024,05-11-2024,06-11-2024,07-11-2024,08-11-2024,09-11-2024,10-11-2024,11-11-2024,12-11-2024,13-11-2024,14-11-2024,15-11-2024,16-11-2024,17-11-2024,18-11-2024,19-11-2024,20-11-2024,21-11-2024,22-11-2024,23-11-2024,24-11-2024,25-11-2024,26-11-2024,1st Quartile,3rd Quartile,IQR,Upper,Lower,Yesterday,last_7_days,last_30_days,% Change Last 7 Days,% Change Last 30 Days
0,2025-02-06,22,2,13,22,,1.0,NaN,CM,COX MOBILE,DR_Mobile Lock Enabled,,Apple iOS,,39.0,37.0,37.0,25.0,46.0,48.0,44.0,37.0,56.0,32.0,36.0,36.0,30.0,40.0,36.0,24.0,37.0,25.0,42.0,33.0,26.0,34.0,35.0,44.0,39.0,42.0,52.0,44.0,45.0,46.0,51.0,35.0,54.0,45.0,43.0,49.0,50.0,41.0,46.0,49.0,49.0,51.0,39.0,44.0,44.0,24.0,43.0,59.0,50.0,46.0,41.0,39.0,43.0,53.0,48.0,40.0,47.0,41.0,43.0,44.0,40.0,42.0,49.0,46.0,54.0,39.0,39.0,36.0,34.0,59.0,45.0,41.0,39.0,39.0,40.0,51.0,47.0,41.0,52.0,38.0,48.0,57.0,46.0,49.0,59.0,51.0,47.0,42.0,61.0,42.0,0.0,143.00,143.00,321.7500,-178.7500,42.0,50.714286,45.833333,-17.183099,-8.363636
1,2025-02-06,23,2,13,23,,1.0,NaN,CM,COX MOBILE,DR_Mobile Lock Disabled,,Apple iOS,,512.0,497.0,446.0,422.0,423.0,531.0,547.0,563.0,516.0,448.0,389.0,562.0,564.0,587.0,597.0,571.0,506.0,481.0,551.0,547.0,582.0,549.0,691.0,600.0,536.0,669.0,659.0,718.0,694.0,673.0,574.0,532.0,620.0,616.0,609.0,669.0,647.0,620.0,500.0,608.0,591.0,597.0,610.0,598.0,533.0,460.0,649.0,633.0,615.0,621.0,608.0,551.0,529.0,652.0,650.0,629.0,640.0,585.0,539.0,474.0,584.0,616.0,591.0,541.0,580.0,493.0,501.0,615.0,529.0,558.0,585.0,562.0,532.0,461.0,570.0,566.0,583.0,636.0,606.0,518.0,461.0,610.0,639.0,656.0,616.0,651.0,627.0,525.0,679.0,648.0,0.0,143.00,143.00,321.7500,-178.7500,648.0,627.571429,572.166667,3.255179,13.253714
2,2025-02-06,74,5,4,74,,1.0,NaN,ECN,EASY CON NECT,DR_Easy Connect - Self install success,,Apple iOS,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,143.00,143.00,321.7500,-178.7500,0.0,NaN,NaN,0,0
3,2025-02-06,75,5,4,75,,1.0,NaN,ECN,EASY CON NECT,DR_Easy Connect - Self install Timeout,,Apple iOS,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,143.00,143.00,321.7500,-178.7500,0.0,NaN,NaN,0,0
4,2025-02-06,10000022,2,13,22,,NaN,NaN,CM,COX MOBILE,DR_Mobile Lock Enabled,,Both,,782.0,790.0,708.0,607.0,663.0,829.0,858.0,856.0,836.0,723.0,609.0,825.0,814.0,879.0,894.0,844.0,745.0,685.0,816.0,804.0,853.0,800.0,993.0,896.0,766.0,937.0,984.0,1025.0,965.0,995.0,880.0,770.0,930.0,972.0,932.0,963.0,973.0,918.0,731.0,890.0,884.0,904.0,931.0,870.0,771

In [90]:
last_7_and_30_days_df.shape

(714, 114)

In [91]:
last_7_and_30_days_df[['1st Quartile', '3rd Quartile', 'IQR','Upper', 'Lower', 'Yesterday', 'last_7_days', 'last_30_days', '% Change Last 7 Days', '% Change Last 30 Days']] = last_7_and_30_days_df[['1st Quartile', '3rd Quartile', 'IQR','Upper', 'Lower', 'Yesterday', 'last_7_days', 'last_30_days', '% Change Last 7 Days', '% Change Last 30 Days']].round(3)

In [92]:
last_7_and_30_days_df.head()

,create_dt,metric_id,feature_id,feature_seq_num,metric_sequence_num,metric_seqno,level_no,parent_id,unique_identifier,feature_name,metrics,display_names,operating_system_type,metric_nature,29-08-2024,30-08-2024,31-08-2024,01-09-2024,02-09-2024,03-09-2024,04-09-2024,05-09-2024,06-09-2024,07-09-2024,08-09-2024,09-09-2024,10-09-2024,11-09-2024,12-09-2024,13-09-2024,14-09-2024,15-09-2024,16-09-2024,17-09-2024,18-09-2024,19-09-2024,20-09-2024,21-09-2024,22-09-2024,23-09-2024,24-09-2024,25-09-2024,26-09-2024,27-09-2024,28-09-2024,29-09-2024,30-09-2024,01-10-2024,02-10-2024,03-10-2024,04-10-2024,05-10-2024,06-10-2024,07-10-2024,08-10-2024,09-10-2024,10-10-2024,11-10-2024,12-10-2024,13-10-2024,14-10-2024,15-10-2024,16-10-2024,17-10-2024,18-10-2024,19-10-2024,20-10-2024,21-10-2024,22-10-2024,23-10-2024,24-10-2024,25-10-2024,26-10-2024,27-10-2024,28-10-2024,29-10-2024,30-10-2024,31-10-2024,01-11-2024,02-11-2024,03-11-2024,04-11-2024,05-11-2024,06-11-2024,07-11-2024,08-11-2024,09-11-2024,10-11-2024,11-11-2024,12-11-2024,13-11-2024,14-11-2024,15-11-2024,16-11-2024,17-11-2024,18-11-2024,19-11-2024,20-11-2024,21-11-2024,22-11-2024,23-11-2024,24-11-2024,25-11-2024,26-11-2024,1st Quartile,3rd Quartile,IQR,Upper,Lower,Yesterday,last_7_days,last_30_days,% Change Last 7 Days,% Change Last 30 Days
0,2025-02-06,22,2,13,22,,1.0,NaN,CM,COX MOBILE,DR_Mobile Lock Enabled,,Apple iOS,,39.0,37.0,37.0,25.0,46.0,48.0,44.0,37.0,56.0,32.0,36.0,36.0,30.0,40.0,36.0,24.0,37.0,25.0,42.0,33.0,26.0,34.0,35.0,44.0,39.0,42.0,52.0,44.0,45.0,46.0,51.0,35.0,54.0,45.0,43.0,49.0,50.0,41.0,46.0,49.0,49.0,51.0,39.0,44.0,44.0,24.0,43.0,59.0,50.0,46.0,41.0,39.0,43.0,53.0,48.0,40.0,47.0,41.0,43.0,44.0,40.0,42.0,49.0,46.0,54.0,39.0,39.0,36.0,34.0,59.0,45.0,41.0,39.0,39.0,40.0,51.0,47.0,41.0,52.0,38.0,48.0,57.0,46.0,49.0,59.0,51.0,47.0,42.0,61.0,42.0,0.0,143.00,143.00,321.750,-178.750,42.0,50.714,45.833,-17.183099,-8.363636
1,2025-02-06,23,2,13,23,,1.0,NaN,CM,COX MOBILE,DR_Mobile Lock Disabled,,Apple iOS,,512.0,497.0,446.0,422.0,423.0,531.0,547.0,563.0,516.0,448.0,389.0,562.0,564.0,587.0,597.0,571.0,506.0,481.0,551.0,547.0,582.0,549.0,691.0,600.0,536.0,669.0,659.0,718.0,694.0,673.0,574.0,532.0,620.0,616.0,609.0,669.0,647.0,620.0,500.0,608.0,591.0,597.0,610.0,598.0,533.0,460.0,649.0,633.0,615.0,621.0,608.0,551.0,529.0,652.0,650.0,629.0,640.0,585.0,539.0,474.0,584.0,616.0,591.0,541.0,580.0,493.0,501.0,615.0,529.0,558.0,585.0,562.0,532.0,461.0,570.0,566.0,583.0,636.0,606.0,518.0,461.0,610.0,639.0,656.0,616.0,651.0,627.0,525.0,679.0,648.0,0.0,143.00,143.00,321.750,-178.750,648.0,627.571,572.167,3.255179,13.253714
2,2025-02-06,74,5,4,74,,1.0,NaN,ECN,EASY CON NECT,DR_Easy Connect - Self install success,,Apple iOS,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,143.00,143.00,321.750,-178.750,0.0,NaN,NaN,0,0
3,2025-02-06,75,5,4,75,,1.0,NaN,ECN,EASY CON NECT,DR_Easy Connect - Self install Timeout,,Apple iOS,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,143.00,143.00,321.750,-178.750,0.0,NaN,NaN,0,0
4,2025-02-06,10000022,2,13,22,,NaN,NaN,CM,COX MOBILE,DR_Mobile Lock Enabled,,Both,,782.0,790.0,708.0,607.0,663.0,829.0,858.0,856.0,836.0,723.0,609.0,825.0,814.0,879.0,894.0,844.0,745.0,685.0,816.0,804.0,853.0,800.0,993.0,896.0,766.0,937.0,984.0,1025.0,965.0,995.0,880.0,770.0,930.0,972.0,932.0,963.0,973.0,918.0,731.0,890.0,884.0,904.0,931.0,870.0,771.0,635.0,904.0,938.0

In [93]:
last_7_and_30_days_df.shape

(714, 114)

In [94]:
import pandas as pd

def add_colour_indicators(df):
    # Function to assign colour_indicator based on the logic
    def assign_colour_indicator(row):
        if row['metric_nature'] == 'Positive':
            if row['Yesterday'] > row['Upper']:
                return 'Positive Above'
            elif row['Yesterday'] < row['Lower']:
                return 'Positive Below'
            else:
                return 'Positive Within'
        elif row['metric_nature'] == 'Negative':
            if row['Yesterday'] > row['Upper']:
                return 'Negative Above'
            elif row['Yesterday'] < row['Lower']:
                return 'Negative Below'
            else:
                return 'Negative Within'
        return None  # In case metric_nature is neither positive nor negative

    # Function to assign colour_indicator1 based on colour_indicator values
    def assign_colour_indicator1(row):
        if row['metric_nature_indicator'] in ['Positive Above', 'Negative Below']:
            return 'Green'
        elif row['metric_nature_indicator'] in ['Positive Below', 'Negative Above']:
            return 'Red'
        elif row['metric_nature_indicator'] in ['Positive Within', 'Negative Within']:
            return 'White'
        return None  # In case colour_indicator is not set
    
    # Apply the function to create the colour_indicator column
    df['metric_nature_indicator'] = df.apply(assign_colour_indicator, axis=1)
    
    # Apply the function to create the colour_indicator1 column
    df['colour_indicator'] = df.apply(assign_colour_indicator1, axis=1)
    
    return df  # Return the modified DataFrame



In [95]:
# Call the function and get the updated DataFrame
colour_indicators_df  = add_colour_indicators(last_7_and_30_days_df)
colour_indicators_df = colour_indicators_df[['create_dt', 'metric_id', 'feature_name', 'metrics', 'display_names', 'operating_system_type', '1st Quartile', '3rd Quartile', 'IQR', 'Upper', 'Lower', 'Yesterday', 'last_7_days', 'last_30_days', '% Change Last 7 Days', '% Change Last 30 Days', 'metric_nature_indicator', 'colour_indicator']]

# Display the resulting dataframe
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 500)
colour_indicators_df.head()


,create_dt,metric_id,feature_name,metrics,display_names,operating_system_type,1st Quartile,3rd Quartile,IQR,Upper,Lower,Yesterday,last_7_days,last_30_days,% Change Last 7 Days,% Change Last 30 Days,metric_nature_indicator,colour_indicator
0,2025-02-06,22,COX MOBILE,DR_Mobile Lock Enabled,,Apple iOS,0.0,143.00,143.00,321.750,-178.750,42.0,50.714,45.833,-17.183099,-8.363636,None,None
1,2025-02-06,23,COX MOBILE,DR_Mobile Lock Disabled,,Apple iOS,0.0,143.00,143.00,321.750,-178.750,648.0,627.571,572.167,3.255179,13.253714,None,None
2,2025-02-06,74,EASY CON NECT,DR_Easy Connect - Self install success,,Apple iOS,0.0,143.00,143.00,321.750,-178.750,0.0,NaN,NaN,0,0,None,None
3,2025-02-06,75,EASY CON NECT,DR_Easy Connect - Self install Timeout,,Apple iOS,0.0,143.00,143.00,321.750,-178.750,0.0,NaN,NaN,0,0,None,None
4,2025-02-06,10000022,COX MOBILE,DR_Mobile Lock Enabled,,Both,805.0,930.75,125.75,1087.938,647.812,1023.0,971.286,868.433,5.324312,17.798334,None,None


In [96]:
#colour_indicators_df.to_csv(r'D:\Healthsore Data\Table4_athena.csv', index = False)
